In [1]:
import numpy as np
import pandas as pd

how to avoid creating unused temporary variables
```python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

 assign method
```python
# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
# It returns a new object rather than modifying the object in-place
df2 = df.assign(k=v)
```

assign enables easier method chaining
```python
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())
```

To show callables in action, consider a fragment of the example from before:

```python
df = load_data()
df2 = df[df['col2'] < 0]
# this can be rewritten as
df = (load_data()
      [lambda x: x['col2'] < 0])
```

write the entire sequences as a single chained expression
```python
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```

## The pipe Method

Sometimes you need to use your own functions or functions from third-party libraries.
```python
# consider a sequence of function calls:
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

When using functions that accept and return Series or DataFrame objects, you can
rewrite this using calls to  pipe :
```python
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

# suppose you want to do the following transformation
```python
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```

Then it is possible to write
```python
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```